In [5]:
BUCKET_NAME = "hassan-ai-platform-temp" #@param {type:"string"}
REGION = "us-west1" #@param {type:"string"}

In [6]:
! gsutil ls -al gs://$BUCKET_NAME

   3974305  2021-08-25T07:43:37Z  gs://hassan-ai-platform-temp/adult.data#1629877417446187  metageneration=1
      4551  2021-08-30T06:10:23Z  gs://hassan-ai-platform-temp/iris.csv#1630303823649923  metageneration=1
      4729  2021-08-26T13:20:42Z  gs://hassan-ai-platform-temp/model.joblib#1629984042859678  metageneration=1
                                 gs://hassan-ai-platform-temp/custom-container-job-dir/
                                 gs://hassan-ai-platform-temp/custom_models/
                                 gs://hassan-ai-platform-temp/keras-job-dir-pred/
                                 gs://hassan-ai-platform-temp/keras-job-dir/
TOTAL: 3 objects, 3983585 bytes (3.8 MiB)


In [3]:
# Clone the repository of AI Platform samples
! git clone --depth 1 https://github.com/GoogleCloudPlatform/cloudml-samples


fatal: destination path 'cloudml-samples' already exists and is not an empty directory.
/home/jupyter/cloudml-samples/census/tf-keras


In [1]:

# Set the working directory to the sample code directory
%cd cloudml-samples/census/tf-keras

/home/jupyter/cloudml-samples/census/tf-keras


In [2]:
# `ls` shows the working directory's contents. The `p` flag adds trailing 
# slashes to subdirectory names. The `R` flag lists subdirectories recursively.
! ls -pR

.:
'=1.16.0'			   local-training-output/	 scripts/
 Custom_routine_Prediction.ipynb   prediction.json		 trainer/
 README.md			   prediction_input.json	 untitled.txt
 hptuning_config.yaml		   requirements.txt
 input.json			   sample_prediction_input.txt

./local-training-output:
keras_export/  keras_tensorboard/

./local-training-output/keras_export:
assets/  saved_model.pb  variables/

./local-training-output/keras_export/assets:
saved_model.json

./local-training-output/keras_export/variables:
checkpoint		       variables.data-00001-of-00002
variables.data-00000-of-00002  variables.index

./local-training-output/keras_tensorboard:
events.out.tfevents.1629878208.python-20210825-131821-hassan
events.out.tfevents.1629878209.python-20210825-131821-hassan.profile-empty
plugins/

./local-training-output/keras_tensorboard/plugins:
profile/

./local-training-output/keras_tensorboard/plugins/profile:
2021-08-25_07-56-49/

./local-training-output/keras_tensorboard/plugins/profile/2021-08-25_07-

In [3]:
! pip install -r requirements.txt

  Using cached numpy-1.18.5-cp37-cp37m-manylinux1_x86_64.whl (20.1 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.21.2
    Uninstalling numpy-1.21.2:
      Successfully uninstalled numpy-1.21.2


In [8]:
# Explicitly tell `gcloud ai-platform local train` to use Python 3 
! gcloud config set ml_engine/local_python $(which python3)

Updated property [ml_engine/local_python].


In [8]:
# Local train
# This is similar to `python -m trainer.task --job-dir local-training-output`
# but it better replicates the AI Platform environment, especially for
# distributed training (not applicable here).
! gcloud ai-platform local train \
  --package-path trainer \
  --module-name trainer.task \
  --job-dir local-training-output

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
2021-08-25 07:56:48.495279: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2021-08-25 07:56:48.500451: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2200155000 Hz
2021-08-25 07:56:48.500778: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x5614860700f0 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2021-08-25 07:56:48.500820: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
Train on 254 steps, validate on 1 steps

Epoch 00001: LearningRateScheduler reducing learning ra

### Submit a Train job AI platform

In [7]:
JOB_NAME = 'my_first_keras_job_test_prediction'
JOB_DIR = 'gs://' + BUCKET_NAME + '/keras-job-dir'

In [ ]:
! gcloud ai-platform jobs submit training $JOB_NAME \
  --package-path trainer/ \
  --module-name trainer.task \
  --region $REGION \
  --python-version 3.7 \
  --runtime-version 1.15 \
  --job-dir $JOB_DIR \
  --stream-logs

# Prediction

In [26]:
MODEL_NAME = "my_first_keras_model1"
REGION = 'global'
! gcloud ai-platform models create $MODEL_NAME \
  --regions $REGION

Using endpoint [https://ml.googleapis.com/]
ERROR: (gcloud.ai-platform.models.create) Resource in projects [qp-acc-aia-migrate-2021-06] is the subject of a conflict: Field: model.name Error: A model with the same name already exists.
- '@type': type.googleapis.com/google.rpc.BadRequest
  fieldViolations:
  - description: A model with the same name already exists.
    field: model.name


In [29]:
KERAS_EXPORT_DIRS = !gsutil ls $JOB_DIR/keras_export/
print(KERAS_EXPORT_DIRS)

['CommandException: "ls" command does not support "file://" URLs. Did you mean to use a gs:// URL?']


In [28]:
KERAS_EXPORT_DIRS[1]

IndexError: list index out of range

In [26]:
MODEL_VERSION = "v1"
REGION ='global'
JOB_DIR = 'gs://' + BUCKET_NAME + '/keras-job-dir'
# Get a list of directories in the `keras_export` parent directory
KERAS_EXPORT_DIRS = ! gsutil ls $JOB_DIR/keras_export/

# Pick the directory with the latest timestamp, in case you've trained
# multiple times
SAVED_MODEL_PATH = KERAS_EXPORT_DIRS[0]

# Create model version based on that SavedModel directory
! gcloud ai-platform versions create $MODEL_VERSION \
  --model $MODEL_NAME \
  --runtime-version 1.15 \
  --python-version 3.7 \
  --framework tensorflow \
  --region $REGION \
  --origin $SAVED_MODEL_PATH

Using endpoint [https://ml.googleapis.com/]
Creating version (this might take a few minutes)......done.                    


In [10]:

from trainer import util

_, _, eval_x, eval_y = util.load_data()

prediction_input = eval_x.sample(20)
prediction_targets = eval_y[prediction_input.index]

prediction_input

,age,workclass,education_num,marital_status,occupation,relationship,race,capital_gain,capital_loss,hours_per_week,native_country
10686,1.995213,5,-0.419365,2,2,0,4,-0.144807,-0.217119,-1.648238,38
4880,0.682402,3,-0.030311,2,3,0,4,-0.144807,-0.217119,0.773054,38
2972,-0.265739,3,1.525903,2,9,0,4,13.275268,-0.217119,0.773054,38
4062,0.609468,3,-2.364632,2,13,0,4,-0.144807,-0.217119,-0.034043,38
7329,0.536534,3,1.136850,5,9,4,2,-0.144807,-0.217119,-0.679721,22
11107,-1.286815,3,-0.419365,4,0,3,4,-0.144807,-0.217119,-0.437592,38
12364,0.390666,3,-0.419365,0,7,4,4,-0.144807,-0.217119,-0.034043,38
2014,-0.776277,3,-1.975579,4,13,1,4,0.774343,-0.217119,-0.437592,38
12968,1.192940,3,-0.030311,2,13,0,4,-0.144807,-0.217119,2.710088,38
1006,-0.411607,3,0.747796,2,11,0,4,-0.144807,-0.217119,0.450215,38


In [11]:
import pandas as pd

_, eval_file_path = util.download(util.DATA_DIR)
raw_eval_data = pd.read_csv(eval_file_path,
                            names=util._CSV_COLUMNS,
                            na_values='?')

raw_eval_data.iloc[prediction_input.index]

,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,gender,capital_gain,capital_loss,hours_per_week,native_country,income_bracket
10686,66,Self-emp-not-inc,331960,HS-grad,9,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,20,United-States,<=50K
4880,48,Private,259873,Some-college,10,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,50,United-States,<=50K
2972,35,Private,151835,Masters,14,Married-civ-spouse,Prof-specialty,Husband,White,Male,99999,0,50,United-States,>50K
4062,47,Private,202395,7th-8th,4,Married-civ-spouse,Transport-moving,Husband,White,Male,0,0,40,United-States,<=50K
7329,46,Private,155509,Bachelors,13,Separated,Prof-specialty,Unmarried,Black,Female,0,0,32,Jamaica,<=50K
11107,21,Private,97214,HS-grad,9,Never-married,Adm-clerical,Own-child,White,Female,0,0,35,United-States,<=50K
12364,44,Private,33895,HS-grad,9,Divorced,Other-service,Unmarried,White,Female,0,0,40,United-States,<=50K
2014,28,Private,107812,9th,5,Never-married,Transport-moving,Not-in-family,White,Male,6849,0,35,United-States,<=50K
12968,55,Private,323639,Some-college,10,Married-civ-spouse,Transport-moving,Husband,White,Male,0,0,74,United-States,>50K
1006,33,Private,172237,Assoc-acdm,12,Married-civ-spouse,Sales,Husband,White,Male,0,0,46,United-States,<=50K


In [1]:
import json

with open('prediction_input.json', 'w') as json_file:
  for row in prediction_input.values.tolist():
    json.dump(row, json_file)
    json_file.write('\n')

! cat prediction_input.json

NameError: name 'prediction_input' is not defined

In [16]:
REGION = 'global'
MODEL_NAME = 'my_first_keras_model1'
MODEL_VERSION = 'v1'
!gcloud ai-platform predict \
  --model $MODEL_NAME \
  --version $MODEL_VERSION \
  --region $REGION \
  --json-instances prediction_input.json

Using endpoint [https://ml.googleapis.com/]
DENSE_4
[0.0753755122423172]
[0.021521825343370438]
[0.0007477122126147151]


In [16]:
! gcloud ai-platform predict --help

NAME
    gcloud ai-platform predict - run AI Platform online prediction

SYNOPSIS
    gcloud ai-platform predict --model=MODEL
        (--json-instances=JSON_INSTANCES | --json-request=JSON_REQUEST
          | --text-instances=TEXT_INSTANCES) [--region=REGION]
        [--signature-name=SIGNATURE_NAME] [--version=VERSION]
        [GCLOUD_WIDE_FLAG ...]

DESCRIPTION
    gcloud ai-platform predict sends a prediction request to AI Platform for
    the given instances. This command will read up to 100 instances, though the
    service itself will accept instances up to the payload limit size
    (currently, 1.5MB). If you are predicting on more instances, you should use
    batch prediction via

        $ gcloud ai-platform jobs submit prediction.

REQUIRED FLAGS
     --model=MODEL
        Name of the model.

     Exactly one of these must be specified:

       --json-instances=JSON_INSTANCES
          Path to a local file from which instances are read. Instances are in
          JSON forma

In [31]:
# Delete model version resource
!gcloud ai-platform versions delete $MODEL_VERSION --quiet --model $MODEL_NAME 

ERROR: (gcloud.ai-platform.versions.delete) argument --model: expected one argument
Usage: gcloud ai-platform versions delete VERSION --model=MODEL [optional flags]
  optional flags may be  --help | --region

For detailed information on this command and its flags, run:
  gcloud ai-platform versions delete --help


In [32]:

# Delete model resource
! gcloud ai-platform models delete $MODEL_NAME --quiet

Using endpoint [https://us-central1-ml.googleapis.com/]
ERROR: (gcloud.ai-platform.models.delete) NOT_FOUND: Field: name Error: The model resource: "my_first_keras_model1" was not found. Please create the model resource first by using 'gcloud ai-platform models create my_first_keras_model1'.
- '@type': type.googleapis.com/google.rpc.BadRequest
  fieldViolations:
  - description: "The model resource: \"my_first_keras_model1\" was not found. Please\
      \ create the model resource first by using 'gcloud ai-platform models create\
      \ my_first_keras_model1'."
    field: name


In [ ]:
# Delete Cloud Storage objects that were created
! gsutil -m rm -r $JOB_DIR

# If the training job is still running, cancel it
! gcloud ai-platform jobs cancel $JOB_NAME --quiet --verbosity critical